<font size = 10>This notebook is a tool to Build a graph for spatio-temporal anomaly detection in 5G networks </font>

`First: we need those import`

In [ ]:
import os
import random
import numpy as np
import pandas as pd
from timeit import default_timer as timer
import time
import re
import Tools_first_step as tfs
import matplotlib.pyplot as plt
import seaborn as sns
pd.options.mode.chained_assignment = None

The threashold is required to select when we want an edge or not using this equation:

$$   
    A_{i,j}= 
    \left\{
    \begin{array}{ll}
    0 & \text{if  ~ } S_{i,j} \leq T \\
    1 & \text{if ~} S_{i,j} > T 
    \end{array}
    \right.
     $$

The L is used in a Lambda model in the equation: $$  S_{i,j}=\lambda |Corr(x_i,x_j)| + (1-\lambda)|Cosim(x_i, x_j)| $$ (ref. in the article XXX)

In [ ]:
threashold=0.5
L=0.7

This is all the path of file we need in this notebook

In [ ]:
path_dataset= '/path/cisco/preprocessed_final_short_cisco_train.csv'
matrix_correlation_path='/path/corr_matrice.csv' #not mandatory, we can also compute directly
dataset="cisco"

> ## I) Load the dataset

In [ ]:
df=pd.read_csv(path_dataset)#, nrows=300) 
#nrows can be used or not
#depending if we need to calculate the correlation matrix or import it
columns=df.columns

In [ ]:
#verify the dataset :
print(df.shape)
col_to_keep=tfs.rmv_Unnamed(df.columns)
df=df[col_to_keep]
print(df.shape)

# II) This part will generate a graph with our data

> ### For this, we need to tokenize the feature name

In [ ]:
name_columns=df.columns
print(tfs.estimate_total_number_of_word(name_columns, dt=dataset))
number_complet=tfs.estimate_total_number_of_word(name_columns, dt=dataset)
print(tfs.find_frequence_of_words(name_columns, dt=dataset)[1])

In [ ]:
tfs.split_all(name_columns, dt=dataset)

In [ ]:
dico_all_word=tfs.find_frequence_of_words(name_columns, dt=dataset)[1]
dico2=tfs.find_frequence_of_words(name_columns, dt=dataset)[0]

In [ ]:
import nltk
if dataset=="5G3E":
    doc_path="truc.txt" #prometheus metric file explained
    new_dico_all_word=tfs.cleaning_txt_documentation(doc_path)
    print(new_dico_all_word[100:103])

In [ ]:
#we need to get every word, including those that are only in the dataset, so we just add the column name at the end of the description file to be sure to have it
if dataset=="5G3E":
    final_dictionnary=new_dico_all_word+tfs.split_all(name_columns, dt=dataset)
else:
    final_dictionnary=tfs.split_all(name_columns, dt=dataset)

> ### Using word2vec, we transform our token, then features names into vectors

Instead, we will probably have to try BERT with the time ! BERT seams better today, but we have to give context for this one

In [ ]:
from gensim.models import KeyedVectors
from gensim.models import Word2Vec

# Load pretrained model (download if needed: GoogleNews-vectors-negative300.bin)
pretrained_model = KeyedVectors.load_word2vec_format('/path/GoogleNews-vectors-negative300.bin', binary=True)

# Convert KeyedVectors to a Word2Vec instance for further training
model = Word2Vec(vector_size=300, min_count=1)
model.build_vocab([list(pretrained_model.key_to_index.keys())], update=False)
model.wv.vectors = pretrained_model.vectors

In [ ]:
model.build_vocab(final_dictionnary, update=True)

In [ ]:
model.train(final_dictionnary, total_examples=len(final_dictionnary), epochs=10)

In [ ]:
a=tfs.number_a_word_in_doc(name_columns, dico2, dico_all_word, dt=dataset)
print(a)

In [ ]:
score_for_each=[]
for word, freq in dico2.items():
    a=tfs.TF_new_version(word, dico2, number_complet)
    score_for_each.append(a)

plt.plot(score_for_each)

> ### Now we need to use a similarity score to evaluate the difference between two features names

In [ ]:

sequence1 =model.wv['infra'] 
sequence2 =model.wv['bgp']
similarity = tfs.cosine_similarity(sequence1, sequence2)
print("Cosine Similarity:", similarity)

In [ ]:
a,b=tfs.score_similarity_current(name_columns, model, dt=dataset)
print(b)

In [ ]:
nodes=name_columns

cosine_similarities, vectores_places=tfs.score_similarity_current(name_columns, model, dt=dataset)

> ### Once this is done, we can plot the first graph

In [ ]:
xe,ye,ze, all_edges=tfs.get_edges_v3(cosine_similarities,threashold, vectores_places, dataset)

In [ ]:
nb=tfs.get_distribution_number_of_edges_per_nodes("Cisco",cosine_similarities,threashold, vectores_places)
#mid distrib not final !

> ### We can even plot part of the graph that contains a specific word (in the dictionnary of word in all the features)

# III) This part will propose to modify the edges selections, by using also the correlation matrix

In [ ]:
corr_mat=df.corr()
#Here we decide to load the correlation instead of compute it

In [ ]:
corr_mat.index=df.columns
corr_mat.head()

In [ ]:
dico_correlation=tfs.change_into_dico(corr_mat)

In [ ]:
print(len(dico_correlation))
print(len(cosine_similarities))
print(len(corr_mat.iloc[1]))

> ### Final score:

The function "Build_new_score_product" produce the score given by the equation: 

$$ S_{i,j}= |Corr(x_i,x_j)|*|Cosim(x_i, x_j)| $$

The If we prefer the Lambda model from the equation:

$$  S_{i,j}=\lambda |Corr(x_i,x_j)| + (1-\lambda)|Cosim(x_i, x_j)|$$

We can use here instead "build_score_lambda"

If we do not want to use a TopK function, we can use get_edges_v3  or simply select a k high enough

In [ ]:
My_dictionary=tfs.build_score_lambda(dico_correlation, cosine_similarities, L) #no plot in >3D warning
my_edges_test= tfs.edges_dico(My_dictionary, threashold)
useless1,useless2,useless2, edges_complet=tfs.get_edges_topK(My_dictionary,threashold,vectores_places,"", 200)

In [ ]:
tfs.plot_only_part_data(My_dictionary,threashold,vectores_places,"Cisco") #server_1 #My_dictionnary

In [ ]:
useless1,useless2,useless2, edges_complet=tfs.get_edges_topK(My_dictionary,threashold,vectores_places,"", topK=300)#My_dictionnary/cos_sim/dico_correlation for just similkarity
med2=tfs.get_medium_number_of_edges(edges_complet)
print(med2)

# IV) This section build the adjacency matrix that we willl use in GNN

In [ ]:
adj_3d=tfs.build_adjacency_matrix_v2(edges_complet[1:],corr_mat)

In [ ]:
adj_3np=np.array(adj_3d)

In [ ]:
#visualize the matrix:

downsampled_data = adj_3np[::1, ::1]  # we can downsample if we prefer

# Plot heatmap
plt.figure(figsize=(10, 8))
sns.heatmap(downsampled_data, cmap='viridis')
plt.title('Adjacency matrix 25D')
plt.show()

> ### We made 2 differents models for words, one words projected in 25 dimensions, the other in 3, now we compare the two matrix

# V) Finaly we have to save/load the matrix

Modify the path for saving in the right place

In [ ]:
#np.savetxt('/path/cosi5_l07_th5_rebuild_short.csv',adj_3np, delimiter=',')

In [ ]:
#Here is how to load after
t1=np.genfromtxt('/path.csv', delimiter=',')
print(t1)